# Plotting COLMAP Data and Camera Poses 

In [1]:
# Demo test scene: Gerrard Hall 
# Image data provided by COLMAP 
# Colmap generated: images, 3D points, cameras

import open3d as o3d
import numpy as np
import plotly.graph_objects as go 
from scipy.spatial.transform import Rotation as R
from colmapParsingUtils import *
import copy

from mpl_toolkits.mplot3d import axes3d
from matplotlib import cm
import matplotlib.pyplot as plt

import tensorflow as tf
import cv2

%load_ext autoreload
%autoreload 2
%autosave 180

# # Limit GPU memory, if running notebook on GPU
# gpus = tf.config.experimental.list_physical_devices('GPU')
# print(gpus)
# if gpus:
#     try:
#         memlim = 2*1024
#         tf.config.experimental.set_virtual_device_configuration(gpus[0], [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=memlim)])
#     except RuntimeError as e:
#         print(e)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


2025-06-10 10:09:12.526591: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-10 10:09:12.574916: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-06-10 10:09:12.586311: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-06-10 10:09:12.590228: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-06-10 10:09:12.625969: I tensorflow/core/platform/cpu_feature_guar

Autosaving every 180 seconds


In [2]:
# Load pose estimations from COLMAP 

images_colmap = read_images_text('/home/daniel-choate/ASAR/s2/SfM/GerrardHall/Project/images.txt')
cameras = read_cameras_text('/home/daniel-choate/ASAR/s2/SfM/GerrardHall/Project/cameras.txt')
pts3d = read_points3D_text('/home/daniel-choate/ASAR/s2/SfM/GerrardHall/Project/points3D.txt')

# print(images[1])
# print(cameras[1])
print(pts3d[50550])
print(pts3d[5].xyz)
print(images_colmap[1].point3D_ids)

Point3D(id=50550, xyz=array([-0.80275081,  0.20678899,  0.45578969]), rgb=array([135, 108,  81]), error=1.1460145273910631, image_ids=array([24, 25, 27, 28]), point2D_idxs=array([ 7160,  8258,  8817, 10120]))
[ 1.31424389 -0.25072281 -1.61803924]
[   -1    -1    -1 ...    -1 30823    -1]


In [3]:
# Extract 3d points from raw file 

pointdata_fn = "/home/daniel-choate/ASAR/s2/SfM/GerrardHall/Project/points3D.txt"
num_pts = 75497 # Gerrard Hall

# points_data = []
scene_pts = np.zeros([num_pts, 3]) 
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:]  # Skip the first three lines
    
    for line in lines:
        elements = line.strip().split()
        
        row = [float(elem) for elem in list(elements)]

        scene_pts[idx,0] = row[1]
        scene_pts[idx,1] = row[2]
        scene_pts[idx,2] = row[3]
        idx += 1

# Filter RGB data into separate numpy array 
rgb_data = np.zeros([num_pts, 3])
idx = 0

with open(pointdata_fn, 'r') as file:
    lines = file.readlines()[3:] # Skip the first three lines 
    for line in lines:
        elements = line.strip().split()
        row = [float(elem) for elem in list(elements)]

        rgb_data[idx,0] = row[4]
        rgb_data[idx,1] = row[5]
        rgb_data[idx,2] = row[6]
        idx += 1

# SCALE COLORS
rgb_data = rgb_data/255
# print(scene_pts[:10])

In [4]:
# Convert COLMAP poses (xyz,quats) to rotm

poses = np.zeros([len(images_colmap),4,4])
images = np.zeros([len(poses),250,250,3])


skip_indices = {56, 58}  # Use a set to store the indices to skip

#loop through <images_from_colmap> to get 3D poses of cameras at each timestamp
# print(len(images_from_colmap))
for n in range(len(images_colmap)):
    if n in skip_indices:
        continue


    # Pull quaternion and translation vector
    qvec = images_colmap[n+1].qvec #raw
    # print(qvec)
    # print(images_colmap[n+1].tvec)
    tvec = images_colmap[n+1].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)

    # NERF SPECIFIC ROTATIONS 
    # c2w[0:3,2] *= -1 # flip the y and z axis
    # c2w[0:3,1] *= -1
    # c2w = c2w[[1,0,2,3],:]
    # c2w[2,:] *= -1 # flip whole world upside down
    


    poses[n] = c2w
    # poses[n] = m
    
    #~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

#GET REST OF PARAMS NEEDED FOR tinyNeRF format~~~~~~~~~~~~~~~~~~~~~~~~~~~~    

#fix order of colors
images[:,:,:,0], images[:,:,:,1] = images[:,:,:,1], images[:,:,:,0]

H,W = images.shape[1:3]
# print(H,W)
testimg, testpose = images[55], poses[55]

focal = cameras[1].params[0] #test- see if same focal length can be shared across all images
focal = focal/12
print(focal)
# print(poses)
#~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

318.8074217375768


In [5]:
# np.save('poses_ghall.npy', poses)

# Plotting 3D pt Cloud

In [6]:
# Plotting 3D points
# BY ITSELF
# Data generated by COLMAP

vis = o3d.visualization.Visualizer()
vis.create_window()

scene_cloud = o3d.geometry.PointCloud()
scene_cloud.points = o3d.utility.Vector3dVector(scene_pts)
scene_cloud.colors = o3d.utility.Vector3dVector(rgb_data)

vis.add_geometry(scene_cloud)

# Set point size (too large as is)
render_option = vis.get_render_option()
render_option.point_size = 2.5

vis.reset_view_point(True)

# Run visualizer 
vis.run()
vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 76
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 76
pci id for fd 76: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# Plotting Poses: 3D coordinate frames 

In [7]:
# Plotting POSES: 3D coordinate frames 
# BY THEMSELVES

# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with pose Axes', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=1.0)
vis.add_geometry(axes)

# Colors for the axes: Red, Green, Blue
colors = [
    [1, 0, 0],  # Red for X-axis
    [0, 1, 0],  # Green for Y-axis
    [0, 0, 1]   # Blue for Z-axis
]

print(colors)

# Loop over each pose
for pose in poses:
    # print(i)
    # print(poses[i])
    
    # Extract position and rotation matrix from the 4x4 matrix
    position = pose[:3, 3]  # Position vector (first 3 elements of the last column)
    rotation_matrix = pose[:3, :3]  # Top-left 3x3 submatrix
    print("Position:\n", position)
    print("Rotation Matrix:\n", rotation_matrix)
    
    # Calculate heading vectors (x, y, z axes)
    headings = np.array([
        rotation_matrix @ np.array([1, 0, 0]),  # X-axis
        rotation_matrix @ np.array([0, 1, 0]),  # Y-axis
        rotation_matrix @ np.array([0, 0, 1])   # Z-axis
    ])
    print("Headings:\n", headings)
    
    # Create and add LineSets for each heading vector with respective color
    for heading, color in zip(headings, colors):
        # Ensure that position and heading are NumPy arrays
        position_np = np.array(position)
        end_point = position_np + heading * 0.5
        
        # Create LineSet
        arrow = o3d.geometry.LineSet()
        points = np.array([position_np, end_point])
        arrow.points = o3d.utility.Vector3dVector(points)
        arrow.lines = o3d.utility.Vector2iVector([[0, 1]])
        
        # Set color
        arrow.colors = o3d.utility.Vector3dVector([color])  # Set color for the single line segment
        
        # Add the arrow to the visualizer
        vis.add_geometry(arrow)

# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


[[1, 0, 0], [0, 1, 0], [0, 0, 1]]
Position:
 [-1.02530593  0.66165033  4.5459197 ]
Rotation Matrix:
 [[-0.90737996  0.11774594  0.40348173]
 [-0.04247161  0.92935999 -0.36672355]
 [-0.41815999 -0.34989412 -0.83828177]]
Headings:
 [[-0.90737996 -0.04247161 -0.41815999]
 [ 0.11774594  0.92935999 -0.34989412]
 [ 0.40348173 -0.36672355 -0.83828177]]
Position:
 [-1.26563469  0.65686955  4.40247888]
Rotation Matrix:
 [[-0.87836644  0.11494937  0.46396017]
 [-0.05607907  0.93916634 -0.33885355]
 [-0.47468678 -0.32365605 -0.81848594]]
Headings:
 [[-0.87836644 -0.05607907 -0.47468678]
 [ 0.11494937  0.93916634 -0.32365605]
 [ 0.46396017 -0.33885355 -0.81848594]]
Position:
 [-1.48847961  0.6519923   4.30700439]
Rotation Matrix:
 [[-0.86390519  0.10174918  0.49326963]
 [-0.08074801  0.93873114 -0.33505762]
 [-0.4971394  -0.32928856 -0.80275866]]
Headings:
 [[-0.86390519 -0.08074801 -0.4971394 ]
 [ 0.10174918  0.93873114 -0.32928856]
 [ 0.49326963 -0.33505762 -0.80275866]]
Position:
 [-1.84924958 

# Plotting 3D Poses and pt Cloud

In [8]:
# Plotting poses and 3D cloud

# STRICTLY WITHIN NOTEBOOK 


# fig = go.Figure()

# # Create a marker for the origin
# fig.add_trace(go.Scatter3d(
#     x=[0], y=[0], z=[0],
#     mode='markers',
#     marker=dict(size=1, color='purple'),
#     name='Origin'
# ))

# # Plot coordinate axes x
# fig.add_trace(go.Scatter3d(
#     x=[0, 1], y=[0, 0], z=[0, 0],
#     mode='lines',
#     line=dict(color='red', width=4),
#     name='X Axis'
# ))
# # Plot coordinate axes y
# fig.add_trace(go.Scatter3d(
#     x=[0, 0], y=[0, 1], z=[0, 0],
#     mode='lines',
#     line=dict(color='green', width=4),
#     name='Y Axis'
# ))
# # Plot coordinate axes z
# fig.add_trace(go.Scatter3d(
#     x=[0, 0], y=[0, 0], z=[0, 1],
#     mode='lines',
#     line=dict(color='blue', width=4),
#     name='Z Axis'
# ))

# # Compute and plot quivers for each pose
# arrow_length = 0.5  # Length of the arrow lines
# colors = ['red', 'green', 'blue']
# for pose in poses:
#     position = pose[:, -1]
#     rotation_matrix = pose[:, :3]
    
#     # Calculate heading vectors
#     headings = np.array([
#         rotation_matrix @ np.array([1, 0, 0]),
#         rotation_matrix @ np.array([0, 1, 0]),
#         rotation_matrix @ np.array([0, 0, 1])
#     ])
    
#     # Add arrows
#     for i, (heading, color) in enumerate(zip(headings, colors)):
#         fig.add_trace(go.Scatter3d(
#             x=[position[0], position[0] + heading[0] * arrow_length],
#             y=[position[1], position[1] + heading[1] * arrow_length],
#             z=[position[2], position[2] + heading[2] * arrow_length],
#             mode='lines+markers',
#             line=dict(color=color, width=4),
#             marker=dict(size=.1, color=color),
#             name=f'Heading {chr(88+i)}'
#         ))

# # ADD 3D Points to the figure 
# fig.add_trace(go.Scatter3d(
#     x=scene_pts[:,0],
#     y=scene_pts[:,1],
#     z=scene_pts[:,2],
#     mode='markers',
#     marker=dict(
#         size=1,  # Marker size
#         color=rgb_data,  # Marker color
#         opacity=0.8  # Marker opacity
#     )
# ))


# # Update layout for better visualization
# fig.update_layout(
#     scene=dict(
#         # xaxis=dict(range=[-10, 10], title='x'),
#         # yaxis=dict(range=[-10, 10], title='y'),
#         # zaxis=dict(range=[0, 20], title='z'),
#     ),
#     title='3D Plot with Axes and Vectors',
#     legend=dict(x=0.8, y=0.5), 
#     width = 1000,
#     height = 1000
# )

# # Show the plot
# fig.show()

In [9]:
# Plotting POSES AND 3D CLOUD 
# Data generated by COLMAP 


# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)
vis.add_geometry(axes)

# Colors for the axes: Red, Green, Blue
colors = [
    [1, 0, 0],  # Red for X-axis
    [0, 1, 0],  # Green for Y-axis
    [0, 0, 1]   # Blue for Z-axis
]

# Loop over each pose
for pose in poses:
    # print(i)
    # print(poses[i])
    
    # Extract position and rotation matrix from the 4x4 matrix
    position = pose[:3, 3]  # Position vector (first 3 elements of the last column)
    rotation_matrix = pose[:3, :3]  # Top-left 3x3 submatrix
    print("Position:\n", position)
    print("Rotation Matrix:\n", rotation_matrix)
    
    # Calculate heading vectors (x, y, z axes)
    headings = np.array([
        rotation_matrix @ np.array([1, 0, 0]),  # X-axis
        rotation_matrix @ np.array([0, 1, 0]),  # Y-axis
        rotation_matrix @ np.array([0, 0, 1])   # Z-axis
    ])
    print("Headings:\n", headings)
    
    # Create and add LineSets for each heading vector with respective color
    for heading, color in zip(headings, colors):
        # Ensure that position and heading are NumPy arrays
        position_np = np.array(position)
        end_point = position_np + heading * 0.5
        
        # Create LineSet
        arrow = o3d.geometry.LineSet()
        points = np.array([position_np, end_point])
        arrow.points = o3d.utility.Vector3dVector(points)
        arrow.lines = o3d.utility.Vector2iVector([[0, 1]])
        
        # Set color
        arrow.colors = o3d.utility.Vector3dVector([color])  # Set color for the single line segment
        
        # Add the arrow to the visualizer
        vis.add_geometry(arrow)

vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()



Position:
 [-1.02530593  0.66165033  4.5459197 ]
Rotation Matrix:
 [[-0.90737996  0.11774594  0.40348173]
 [-0.04247161  0.92935999 -0.36672355]
 [-0.41815999 -0.34989412 -0.83828177]]
Headings:
 [[-0.90737996 -0.04247161 -0.41815999]
 [ 0.11774594  0.92935999 -0.34989412]
 [ 0.40348173 -0.36672355 -0.83828177]]
Position:
 [-1.26563469  0.65686955  4.40247888]
Rotation Matrix:
 [[-0.87836644  0.11494937  0.46396017]
 [-0.05607907  0.93916634 -0.33885355]
 [-0.47468678 -0.32365605 -0.81848594]]
Headings:
 [[-0.87836644 -0.05607907 -0.47468678]
 [ 0.11494937  0.93916634 -0.32365605]
 [ 0.46396017 -0.33885355 -0.81848594]]
Position:
 [-1.48847961  0.6519923   4.30700439]
Rotation Matrix:
 [[-0.86390519  0.10174918  0.49326963]
 [-0.08074801  0.93873114 -0.33505762]
 [-0.4971394  -0.32928856 -0.80275866]]
Headings:
 [[-0.86390519 -0.08074801 -0.4971394 ]
 [ 0.10174918  0.93873114 -0.32928856]
 [ 0.49326963 -0.33505762 -0.80275866]]
Position:
 [-1.84924958  0.64409257  4.10018678]
Rotation 

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# New method for pose plotting

In [10]:
def create_coord_frame(trans, dir, size):
    """ 
    creating a coordinate frame for plotting in open3d
    Inputs: translation vector (origin)
    Direction: rotation matrix
    Outputs: 3 arrows representing x,y,z
    """
    red = [1,0,0]
    green = [0,1,0]
    blue = [0,0,1]
    n = np.deg2rad(90)

    origin = [0.,0.,0]

    # Create initial vector 
    # Note, the arrow function begins at the z-axis 
    zcoord = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.02*size, cone_radius=0.03*size, cylinder_height=.4*size, cone_height=0.05*size,
        resolution=20, cylinder_split=4, cone_split=1) 
    
    zcoord.translate(trans)
    zcoord.rotate(dir, center=trans)
    zcoord.paint_uniform_color(blue)

    # Create x-axis
    # Rotate 90 degrees about y to get x-axis from z-axis
    xcoord = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.02*size, cone_radius=0.03*size, cylinder_height=.4*size, cone_height=0.05*size,
        resolution=20, cylinder_split=4, cone_split=1) 
    euler_angles = [0., n, 0.] #rotation about xaxis, yaxis, zaxis
    rotm = R.from_euler('xyz', euler_angles).as_matrix()
    print(rotm)
    # print("\n rotation matrix (3x3) \n", rotm, "\n")
    xcoord.rotate(rotm, center=origin)
    xcoord.translate(trans)
    # dir_inv = np.linalg.inv(dir)
    # total_rot_x = dir_inv*rotm*dir
    # xcoord.rotate(total_rot_x, center=trans)
    xcoord.rotate(dir, center=trans)
    xcoord.paint_uniform_color(red)


    
    # Create y-axis 
    # Rotate 270 degrees about x to get y-axis from z-axis
    ycoord = o3d.geometry.TriangleMesh.create_arrow(
        cylinder_radius=0.02*size, cone_radius=0.03*size, cylinder_height=.4*size, cone_height=0.05*size,
        resolution=20, cylinder_split=4, cone_split=1)
    euler_angles = [n*3, 0., 0.] #rotation about xaxis, yaxis, zaxis
    rotm = R.from_euler('xyz', euler_angles).as_matrix()
    print(rotm)
    ycoord.rotate(rotm, center=origin)
    ycoord.translate(trans)
    # dir_inv = np.linalg.inv(dir)
    # total_rot_y = dir_inv*rotm*dir
    # ycoord.rotate(total_rot_y, center=trans)
    ycoord.rotate(dir, center=trans)
    
    ycoord.paint_uniform_color(green)


    
    return xcoord, ycoord, zcoord


In [11]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Pose Axes Experimenting', width=1000, height=1000)

# Create initial axes 
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=np.array([0.,0.,0]))

# Loop through each pose to gather proper rotation and translation 
for pose in poses:
    # print(pose)
    # Gather top 3x3 rotation matrix
    rotation_m = pose[:3,:3]
    # print(rotation_m)
    # Gather translation vector 
    trans_v = pose[:3,-1]
    # print(trans_v)
    size = 1
    xcoord, ycoord, zcoord = create_coord_frame(trans_v,rotation_m,size)
    vis.add_geometry(xcoord)
    vis.add_geometry(ycoord)
    vis.add_geometry(zcoord)


vis.add_geometry(axes)
# Run the visualizer
vis.run()
vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.0000

# Pose coordinate frames and 3D points

In [12]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='Pose Axes Experimenting', width=1000, height=1000)

# Create initial axes 
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=np.array([0.,0.,0]))

# Loop through each pose to gather proper rotation and translation 
for pose in poses:
    # print(pose)
    # Gather top 3x3 rotation matrix
    rotation_m = pose[:3,:3]
    # print(rotation_m)
    # Gather translation vector 
    trans_v = pose[:3,-1]
    # print(trans_v)
    size = 0.5
    xcoord, ycoord, zcoord = create_coord_frame(trans_v,rotation_m,size)
    vis.add_geometry(xcoord)
    vis.add_geometry(ycoord)
    vis.add_geometry(zcoord)


vis.add_geometry(axes)

# Add 3D points 
vis.add_geometry(scene_cloud)

# Run the visualizer
vis.run()
vis.destroy_window()


libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00]
 [-1.00000000e+00  0.00000000e+00  2.22044605e-16]]
[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00]
 [ 0.00000000e+00 -2.22044605e-16  1.00000000e+00]
 [ 0.00000000e+00 -1.00000000e+00 -2.22044605e-16]]
[[ 2.22044605e-16  0.00000000e+00  1.00000000e+00]
 [ 0.00000000e+00  1.0000

# Using open3D's method for coordinate frame transformations

In [13]:
# Create Open3D visualizer object
vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D Plot with Pose Axes and Point Cloud', width=1000, height=1000)

# Add coordinate axes
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5)

# loop through poses 
for pose in poses:
    homog_t = pose
    axes1 = copy.deepcopy(axes).transform(homog_t)
    vis.add_geometry(axes1)
    

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)

# Size options (jupyter gives issues when running this multiple times, but it looks better)
render_option = vis.get_render_option()
render_option.point_size = 2

# Run the visualizer
vis.run()
vis.destroy_window()

libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
using driver i915 for 75
pci id for fd 75: 8086:a7a0, driver iris
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
libGL: Can't open configuration file /etc/drirc: No such file or directory.
libGL: Can't open configuration file /home/daniel-choate/.drirc: No such file or directory.
Using DRI3 for screen 0


# Attempting point subset plotting from an individual camera pose 

In [ ]:
# Choose an image id
image_id = 5
# Gather 3d point ids from image data 
im_Ids_o = images_colmap[image_id].point3D_ids
# print(images_colmap[image_id])
# print(im_Ids_o)
# print(im_Ids_o[1])
# Loop through and remove null 3d Ids
im_Ids = []
for i in range(len(im_Ids_o)):
    if im_Ids_o[i] > 0:
        im_Ids.append(im_Ids_o[i])

# print(im_Ids)
# Gather xyz points for each 3d point ID
camera_pts_xyz = np.zeros((len(im_Ids),3))
# print(len(camera_pts_xyz))
# print(len(im_Ids))
for i in range(len(im_Ids)):
    camera_pts_xyz[i] = pts3d[im_Ids[i]].xyz

# Get camera pose for chosen image 
def get_pose_id(id):
    qvec = images_colmap[id].qvec
    tvec = images_colmap[n+1].tvec[:,None]
    # print(tvec)
    
    t = tvec.reshape([3,1])
    # print(tvec)
    Rotmat = qvec2rotmat(-qvec)
    
    bottom = np.array([0.0, 0.0, 0.0, 1.0]).reshape([1, 4])
    m = np.concatenate([np.concatenate([Rotmat, t], 1), bottom], 0)
    c2w = np.linalg.inv(m)
    return c2w


homog_t = get_pose_id(image_id)
print(homog_t)
rotation_m = homog_t[:3,:3]
# print(rotation_m)
# Gather translation vector 
trans_v = homog_t[:3,-1]
# print(trans_v)
size = 1

print(camera_pts_xyz)
# print(pts3d[im_Ids[0]].xyz)
# print(pts3d[26358].xyz)
# print(len(images_colmap))
# print(len(images_colmap[1].point3D_ids))

In [ ]:
# Plotting points with regular point cloud 

vis = o3d.visualization.Visualizer()
vis.create_window(window_name='3D points from specified image id', width=1000, height=1000)

# Create initial axes 
axes = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.5, origin=np.array([0.,0.,0]))

camera_specific = o3d.geometry.PointCloud()
camera_specific.points = o3d.utility.Vector3dVector(camera_pts_xyz)
# camera_specific.colors = o3d.utility.Vector3dVector('blue') # Setting blue for now to stand out
camera_specific.paint_uniform_color([0, 0, 1])

xcoord, ycoord, zcoord = create_coord_frame(trans_v,rotation_m,size)

vis.add_geometry(axes)
vis.add_geometry(scene_cloud)
vis.add_geometry(camera_specific)

vis.add_geometry(xcoord)
vis.add_geometry(ycoord)
vis.add_geometry(zcoord)

# render_option = vis.get_render_option()

# render_option.background_color = [1, 1, 1] #[0.678, 0.847, 0.902] # Light blue

vis.run()

vis.destroy_window()
